# Generation of Training Data for Neural Network

This file generates `nn_X.csv` and `nn_Y.csv`.

## Generation of `nn_X.csv`
1. Iterate through some folders (image chains) in `data/images` and put them through the ORB/RANSAC pipeline.
2. Write the incremental transformations calculated in `nn_X.csv`.
3. Read the corresponding values from `data/range.csv` and add them to `nn_X.csv`.

## Generation of `nn_Y.csv`
1. For each image chain that was processed when generating `nn_X.csv`, calculate the incremental transformations using `data/train_labels.csv`.
2. Write the incremental transformations to `nn_Y.csv`.